In [3]:
import sys
sys.path

['/home/sbc_dev/.local/lib/python3.7/site-packages',
 '/mnt/data/sbc_dev/notebooks/LMS/msrank_10k',
 '/usr/local/lib/python37.zip',
 '/usr/local/lib/python3.7',
 '/usr/local/lib/python3.7/lib-dynload',
 '',
 '/home/sbc_dev/.local/lib/python3.7/site-packages',
 '/usr/local/lib/python3.7/site-packages',
 '/home/sbc_dev/.local/lib/python3.7/site-packages/IPython/extensions',
 '/home/sbc_dev/.ipython']

In [8]:
!pip3.7 install -U sklearn -t /home/sbc_dev/.local/lib/python3.7/site-packages

     |████████████████████████████████| 22.3 MB 11.6 MB/s eta 0:00:01
  Using cached numpy-1.21.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
     |████████████████████████████████| 303 kB 55.8 MB/s eta 0:00:01
  Using cached scipy-1.7.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (28.5 MB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=866f10c7365fcb31bf4427108d93ee4c1d304ed0ceb9ab2a4e5fa11bcbbc670c
  Stored in directory: /home/sbc_dev/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
ERROR: deeppavlov 0.8.0 has requirement numpy==1.18.0, but you'll have numpy 1.21.0 which is incompatible.
ERROR: deeppavlov 0.8.0 has requirement pandas==0.25.3, but you'll have pandas 1.3.0 which is incompatible.
ERROR: deeppavlov 0.8.0 has requirement requests==2.22.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: deeppavlov 0.8.0 has requirement scikit-lear

In [421]:
import math
from math import log2

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from catboost.datasets import msrank_10k
from sklearn.preprocessing import StandardScaler, MaxAbsScaler

from typing import List, Union, Dict

In [476]:
class ListNet(torch.nn.Module):
    def __init__(self, num_input_features: int, hidden_dim: int):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.number_of_classes = 1

        fc_layers = nn.ModuleList([
        nn.Sequential(nn.Linear(num_input_features, self.hidden_dim),
                      nn.Dropout(0.1), nn.ReLU()),
#         nn.Sequential(nn.Linear(self.hidden_dim, self.hidden_dim),
#                       nn.Dropout(0.1), nn.ReLU()),
#         nn.Sequential(nn.Linear(self.hidden_dim, self.hidden_dim),
#                       nn.Dropout(0.1), nn.ReLU()),            
        nn.Sequential(nn.Linear(self.hidden_dim, self.number_of_classes))
        ])
        self.fc_layers = fc_layers
        
    def _get_nn_output(self, shape: tuple) -> Union[torch.Tensor, torch.Size]:

        input  = torch.rand(shape) 
        output = input
        # forward pass through conv layers
        for i in range(len(self.fc_layers)):
            output = self.fc_layers[i](output)
            print('nn({0}) shape:{1}'.format(i,np.shape(output)))
                
        output = output.view(output.size(0), -1)
        print('output reshape: {}'.format(np.shape(output)))
        n_size = output.size(1)

        return output, n_size        
        
    def forward(self, input_1: torch.Tensor) -> torch.Tensor:
        # forward pass through fc layers
        output = input_1
        for i in range(len(self.fc_layers)):
            output = self.fc_layers[i](output)

        return output

In [477]:
class Solution:
    def __init__(self, n_epochs: int = 5, listnet_hidden_dim: int = 30,
                 lr: float = 0.001, ndcg_top_k: int = 10):
        
        self.eps = 1.e-10        
        self.std_sc = StandardScaler()
        self.max_sc = MaxAbsScaler() 
        
        self._prepare_data()
        self.listnet_hidden_dim = listnet_hidden_dim
        self.num_input_features = self.X_train.shape[1]
        self.number_of_classes = len(set(self.ys_train))
        self.ndcg_top_k = ndcg_top_k
        self.n_epochs = n_epochs
        self.gain_scheme = 'exp2'

        self.model = self._create_model(
            self.num_input_features, self.listnet_hidden_dim)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

    def _groups_count_vectorizer(self, inp_query_ids: np.ndarray) -> Dict:
        uniq_indx={}
        for ind in inp_query_ids:
            if ind not in uniq_indx:
                uniq_indx[ind]=1
            else:
                uniq_indx[ind]+=1
                
        return uniq_indx
        
    def _get_data(self) -> List[np.ndarray]:
        train_df, test_df = msrank_10k()

        X_train = train_df.drop([0, 1], axis=1).values
        y_train = train_df[0].values
        query_ids_train = train_df[1].values.astype(int)

        X_test = test_df.drop([0, 1], axis=1).values
        y_test = test_df[0].values
        query_ids_test = test_df[1].values.astype(int)

        return [X_train, y_train, query_ids_train, X_test, y_test, query_ids_test]

    def _prepare_data(self) -> None:
        (X_train, y_train, self.query_ids_train,
            X_test, y_test, self.query_ids_test) = self._get_data()
        
        self.X_train = torch.FloatTensor(self._scale_features_in_query_groups(X_train, 
                                                            self.query_ids_train))
        self.X_test  = torch.FloatTensor(self._scale_features_in_query_groups(X_test, 
                                                            self.query_ids_test))
        
        self.ys_train = torch.FloatTensor(y_train)
        self.ys_test  = torch.FloatTensor(y_test)
        
        
    def _scale_features_in_query_groups(self, inp_feat_array: np.ndarray,
                                        inp_query_ids: np.ndarray) -> np.ndarray:
      
        uniq_indx =self._groups_count_vectorizer(inp_query_ids)

        shift = 0
        for ind, cnt in uniq_indx.items():

            sample = inp_feat_array[shift: shift+cnt]
            std_lst = sample.std(axis=0)
            const_cols = np.where(abs(std_lst) <= self.eps)[0]

            mask = np.zeros(sample.shape[-1], dtype=bool)
            mask[const_cols] = True   
            sample[:, mask] = self.max_sc.fit_transform(sample[:,mask])

            if len(const_cols) < sample.shape[-1]:
                mask = np.ones(sample.shape[-1], dtype=bool)
                mask[const_cols] = False  
                sample[:, mask] = self.std_sc.fit_transform(sample[:,mask])

            inp_feat_array[shift: shift+cnt] = sample
            
            shift+=cnt
                                                            
        return inp_feat_array 
 

    def _create_model(self, listnet_num_input_features: int,
                      listnet_hidden_dim: int) -> torch.nn.Module:
        torch.manual_seed(0)
        net = ListNet(listnet_num_input_features, listnet_hidden_dim)
        return net        
    
    def compute_gain(self, y_value: float, gain_scheme: str) -> float:
        if gain_scheme == 'const':
            return y_value
        elif gain_scheme == 'exp2':
            return 2.**y_value - 1.
        else:
            return y_value
    
    def dcg(self, ys_true: torch.Tensor, ys_pred: torch.Tensor,
                ndcg_top_k: int, gain_scheme: str) -> float:
 
        input_shape = ys_pred.shape[0]
        cat = torch.hstack((ys_pred,ys_true))
        t_sorted = cat[cat[:, 0].sort(descending=True)[1]]
    #     t_sorted = t_sorted.unsqueeze(0)

        input_tensor = t_sorted[:ndcg_top_k].numpy()

        factors = torch.Tensor([
                                self.compute_gain(y_value = float(input_tensor[i,1]), 
                                             gain_scheme = gain_scheme)/log2(i+2)
                                for i in range(input_tensor.shape[0])
                               ]
                              ).type(torch.float64)

        return float(torch.cumsum(factors,dim=0)[-1].numpy())    

    def _ndcg_k(self, ys_true: torch.Tensor, ys_pred: torch.Tensor,
                ndcg_top_k: int) -> float:
            
        _dcg = self.dcg(ys_true, ys_pred, ndcg_top_k, self.gain_scheme)

        cat = torch.hstack((ys_pred,ys_true))
        t_sorted = cat[cat[:, 1].sort(descending=True)[1]]

        input_tensor = t_sorted[:ndcg_top_k].numpy()

        factors = torch.Tensor([
                                self.compute_gain(y_value = input_tensor[i,1], 
                                             gain_scheme = self.gain_scheme)/log2(i+2)
                                for i in range(input_tensor.shape[0])
                               ]
                              )    

        idcg = torch.cumsum(factors,dim=0)[-1].numpy() 

        if idcg != 0.:
            res = _dcg/idcg
        else:
            res= 0.

        return res      
    
    def _calc_loss(self, batch_ys: torch.FloatTensor,
                   batch_pred: torch.FloatTensor) -> torch.FloatTensor:
        
        eps = 1.e-14
        y_pred = batch_pred.clone()
        y_true = batch_ys.clone()

#         mask = y_true == padded_value_indicator
#         y_pred[mask] = float('-inf')
#         y_true[mask] = float('-inf')

        preds_smax = F.softmax(y_pred, dim=0)
        true_smax = F.softmax(y_true, dim=0)

        preds_smax = preds_smax + eps
        preds_log = torch.log(preds_smax)

        return -torch.sum(true_smax * preds_log, dim=0)   
    
    def fit(self) -> List[float]:
        
        mean_ndcg_arr = []
        
        for epoch in range(self.n_epochs):

            print("## epoch -> {}".format(epoch))
            self._train_one_epoch()
            curr_ndcg = self._eval_test_set()
            mean_ndcg_arr.append(curr_ndcg)
            
        return mean_ndcg_arr


    def _train_one_epoch(self) -> None:
        
        uniq_indx = uniq_indx = self._groups_count_vectorizer(self.query_ids_train)

        shift = 0
        for ind, cnt in uniq_indx.items():
            
            self.optimizer.zero_grad()
            batch_x = self.X_train[shift: shift+cnt]
            batch_y = self.ys_train[shift: shift+cnt]
                        
            # perform rows shuffling within one batch
            idx = torch.randperm(batch_x.shape[0])
            batch_x = batch_x[idx]
            batch_y = batch_y[idx].view(-1,1)            
            
            batch_pred = self.model(batch_x)
            
            batch_loss = self._calc_loss(batch_y, batch_pred)
            #print("## group num: {}; loss val: {}".format(ind, batch_loss))
            batch_loss.backward(retain_graph=False)
            self.optimizer.step()
            
            shift+=cnt
             
    def _eval_test_set(self) -> float:
        
        uniq_indx = uniq_indx = self._groups_count_vectorizer(self.query_ids_test)
        with torch.no_grad(): 
            
            shift = 0
            ndcgs = []
            for ind, cnt in uniq_indx.items():

                self.optimizer.zero_grad()
                batch_x = self.X_test[shift: shift+cnt]
                batch_y = self.ys_test[shift: shift+cnt].view(-1,1)
                valid_pred = self.model(batch_x)                 
                ndcg_score = self._ndcg_k(batch_y, valid_pred, self.ndcg_top_k)            
                ndcgs.append(ndcg_score)

                shift+=cnt
            
            res = np.mean(ndcgs)
            print("## mean ndcg per group: {}".format(res))
                
            return res    
   

In [478]:
sol = Solution()

In [479]:
out = sol.fit()

## epoch -> 0
## mean ndcg per group: 0.40425502109438227
## epoch -> 1
## mean ndcg per group: 0.43030293840910244
## epoch -> 2
## mean ndcg per group: 0.4294680735740625
## epoch -> 3
## mean ndcg per group: 0.4331150825476493
## epoch -> 4
## mean ndcg per group: 0.4328602619791688


## EDA

In [373]:
train_df, test_df = msrank_10k()

X_train = train_df.drop([0, 1], axis=1).values
y_train = train_df[0].values
query_ids_train = train_df[1].values.astype(int)

X_test = test_df.drop([0, 1], axis=1).values
y_test = test_df[0].values
query_ids_test = test_df[1].values.astype(int)

In [374]:
np.unique(y_train)

array([0., 1., 2., 3., 4.])

In [57]:
train_df.shape

(10000, 138)

In [285]:
train_df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,2.0,1,3,3,0,0,3,1.0,1.000000,0.000000,...,62,11089534,2,116,64034,13,3,0,0,0.000000
1,2.0,1,3,0,3,0,3,1.0,0.000000,1.000000,...,54,11089534,2,124,64034,1,2,0,0,0.000000
2,0.0,1,3,0,2,0,3,1.0,0.000000,0.666667,...,45,3,1,124,3344,14,67,0,0,0.000000
3,2.0,1,3,0,3,0,3,1.0,0.000000,1.000000,...,56,11089534,13,123,63933,1,3,0,0,0.000000
4,1.0,1,3,0,3,0,3,1.0,0.000000,1.000000,...,64,5,7,256,49697,1,13,0,0,0.000000
5,1.0,1,3,0,3,0,3,1.0,0.000000,1.000000,...,62,6,7,210,49923,5,15,0,0,0.000000
6,1.0,1,3,0,3,0,3,1.0,0.000000,1.000000,...,62,6,7,256,50023,3,14,0,0,0.000000
7,2.0,1,3,0,3,0,3,1.0,0.000000,1.000000,...,73,0,17,115,63318,1,3,0,0,0.000000
8,1.0,1,3,0,3,0,3,1.0,0.000000,1.000000,...,51,11089534,15,124,63363,1,3,0,0,0.000000
9,0.0,1,3,0,3,0,3,1.0,0.000000,1.000000,...,48,1,1,541,63363,1,5,0,0,0.000000


In [264]:
std_sc = StandardScaler()
max_sc = MaxAbsScaler()

In [301]:
uniq_indx={}
for ind in query_ids_train:
    if ind not in uniq_indx:
        uniq_indx[ind]=1
    else:
        uniq_indx[ind]+=1
uniq_indx

{1: 86,
 16: 106,
 31: 92,
 46: 120,
 61: 59,
 76: 45,
 91: 74,
 106: 23,
 121: 54,
 136: 172,
 151: 124,
 166: 77,
 181: 77,
 196: 308,
 211: 95,
 226: 126,
 241: 105,
 256: 227,
 271: 81,
 286: 18,
 301: 61,
 316: 95,
 331: 91,
 346: 97,
 361: 81,
 376: 156,
 391: 51,
 406: 196,
 421: 224,
 436: 122,
 451: 56,
 466: 57,
 481: 152,
 496: 89,
 511: 78,
 526: 141,
 541: 233,
 556: 230,
 571: 200,
 586: 117,
 601: 75,
 616: 284,
 631: 133,
 646: 116,
 661: 123,
 676: 163,
 691: 304,
 706: 67,
 721: 137,
 736: 89,
 751: 18,
 766: 206,
 781: 57,
 796: 70,
 811: 76,
 826: 59,
 841: 142,
 856: 87,
 871: 139,
 886: 46,
 901: 280,
 916: 20,
 931: 124,
 946: 85,
 961: 108,
 976: 90,
 991: 115,
 1006: 174,
 1021: 175,
 1036: 103,
 1051: 128,
 1066: 77,
 1081: 129,
 1096: 103,
 1111: 83,
 1126: 165,
 1141: 67,
 1156: 146,
 1171: 54,
 1186: 77,
 1201: 67,
 1216: 71,
 1231: 156,
 1246: 133,
 1261: 107,
 1276: 52,
 1291: 124}

In [299]:
sum([v for k,v in uniq_indx.items()])

10000

In [325]:
eps = 1.e-10
shift = 0
for ind, cnt in uniq_indx.items():
    
    sample = X_train[shift: shift+cnt]
    std_lst = sample.std(axis=0)
    const_cols = np.where(abs(std_lst) <= eps)[0]
    
    mask = np.zeros(sample.shape[-1], dtype=bool)
    mask[const_cols] = True   
    sample[:, mask] = max_sc.fit_transform(sample[:,mask])
    
    if len(const_cols) < sample.shape[-1]:
        mask = np.ones(sample.shape[-1], dtype=bool)
        mask[const_cols] = False  
        sample[:, mask] = std_sc.fit_transform(sample[:,mask])
    
    X_train[shift: shift+cnt] = sample
    shift+=cnt

In [313]:
means = sample.mean(axis=0)
means

array([ 1.74279196e-17,  1.35550486e-17,  2.32372261e-17, -2.19462691e-17,
        1.74279196e-17, -2.84010541e-17,  4.13106242e-17, -4.51834952e-17,
        2.58191401e-18, -2.84010541e-17,  1.93643551e-17,  4.51834952e-18,
        4.51834952e-18,  2.45281831e-17,  2.58191401e-18,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        5.80930652e-18, -3.09829681e-17,  6.45478503e-19,  1.67824411e-17,
       -1.42005271e-17, -2.87237934e-17,  7.74574203e-18,  4.13106242e-17,
       -2.58191401e-18,  8.51224775e-18, -1.93643551e-17, -4.13106242e-17,
        2.25917476e-18, -4.77654092e-17, -3.55013176e-18, -6.45478503e-18,
        2.25917476e-17, -9.35943829e-18, -2.58191401e-17, -6.45478503e-18,
        1.93643551e-18,  1.61369626e-17, -1.45232663e-18,  1.29095701e-18,
       -2.74328364e-18,  4.84108877e-18, -3.74377532e-17, -1.09731345e-17,
       -2.45281831e-17, -7.10026353e-18, -1.03276560e-17, -6.26114148e-17,
       -1.12958738e-18, -

In [274]:
np.where(means==-1)

(array([ 15,  19,  49,  50,  55,  60,  70,  75,  79,  85,  89,  90,  91,
        109, 110, 113, 114, 115, 117, 119, 120, 121, 122]),)

In [282]:
ref = X_train[query_ids_train[query_ids_train==1] - 1,19]
ref

array([6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551, 6.926551, 6.926551, 6.926551, 6.926551,
       6.926551, 6.926551])

In [278]:
sample[:,16]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

In [45]:
res = train_df.groupby([1,]).agg({1:'count'}).rename(columns={1:'cnt'}) \
              .sort_values(by='cnt',ascending=False).reset_index().rename(columns={1:'ids'})

In [46]:
res.head(20)

,ids,cnt
0,196,308
1,691,304
2,616,284
3,901,280
4,541,233
5,556,230
6,256,227
7,421,224
8,766,206
9,571,200
